# Catanomics - Computer Vision and pre-processing
The goal of this notebook is to read in an image of a catan board without number or settlements, and draw the board (ports not included) over the image

In [1]:
# Import necessary libraries
import cv2
import numpy as np

In [2]:
# Show image function for later use
def showImage(img, name=None):
    if not name:
        cv2.imshow("Image display", img)
    else:
        cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Save image function
def saveImage(filename, img, dir):
    # Get full path
    full_path = f"{dir}/{filename}"
    cv2.imwrite(full_path, img)
    print(f"Image saved to {full_path}")

## 1. Read in the image, detect the border, and find the max/min y point of the permiter

In [3]:
# Read in an image
image = cv2.imread('../images/v3/board04.jpg')
# The input imgs are too big, so reduce to 25%
image = cv2.resize(image, (0,0), fx=.25, fy=.25)
showImage(image)
og_img = image.copy()

2024-02-14 18:21:53.646 Python[19928:26746141] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


In [4]:
# Convert the image to hsv
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [5]:
# Define color bounds for the perimiter of the board
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# Create a binary mask where the blue regions are white, and everything else is black
mask = cv2.inRange(hsv_image, lower_blue, upper_blue)

In [6]:
# Bitwise AND to keep the blue parts of the image
blue_regions = cv2.bitwise_and(image, image, mask=mask)

In [7]:
# Perform canny edge detection on the blue_regions

# Convert to grayscale
gray_blue_region = cv2.cvtColor(blue_regions, cv2.COLOR_BGR2GRAY)
# Apply Gaussian Blur
blurred_blue_regions = cv2.GaussianBlur(gray_blue_region, (5,5), 0)
# Canny edge detection
edges = cv2.Canny(blurred_blue_regions, 50, 150)
# Show the result
showImage(edges, "Edges in blue regions")

## 2. Perform Hough Transform on the images

In [8]:
# Perform Hough Transform using `HoughLines`
lines = []
thresh = 200
while len(lines) < 10:
    lines = cv2.HoughLines(edges, 1, np.pi / 180, thresh)
    print(f"Trying threshold {thresh} | Got {len(lines)} lines.")
    if len(lines) >= 10:
        break
    else:
        thresh -= 5

print(f"Results: {thresh} threshold | {len(lines)} lines")

Trying threshold 200 | Got 2 lines.
Trying threshold 195 | Got 2 lines.
Trying threshold 190 | Got 2 lines.
Trying threshold 185 | Got 2 lines.
Trying threshold 180 | Got 3 lines.
Trying threshold 175 | Got 4 lines.
Trying threshold 170 | Got 4 lines.
Trying threshold 165 | Got 4 lines.
Trying threshold 160 | Got 5 lines.
Trying threshold 155 | Got 6 lines.
Trying threshold 150 | Got 7 lines.
Trying threshold 145 | Got 8 lines.
Trying threshold 140 | Got 8 lines.
Trying threshold 135 | Got 9 lines.
Trying threshold 130 | Got 9 lines.
Trying threshold 125 | Got 9 lines.
Trying threshold 120 | Got 9 lines.
Trying threshold 115 | Got 9 lines.
Trying threshold 110 | Got 10 lines.
Results: 110 threshold | 10 lines


In [9]:
# Draw lines on the image
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    
showImage(image)
saveImage("perimiter00.jpg", image, "../images/perimiter/v1")

Image saved to ../images/perimiter/v1/perimiter00.jpg


In [10]:
print(lines[0, 0, 0])

875.0


In [11]:
min_rho_difference = 25
overlapping_indexes = []

for i in range(len(lines)):
    rho, _ = lines[i, 0]
    for j in range(i+1, len(lines)):
        rho2, _ = lines[j, 0]
        if abs(rho - rho2) < min_rho_difference:
            print(f"deleting an overlapping line at index {j}: {lines[j]}")
            overlapping_indexes.append(j)
print(f"Overlapping indexes: {overlapping_indexes}")

deleting an overlapping line at index 4: [[466.         2.146755]]
deleting an overlapping line at index 9: [[141.          0.2268928]]
deleting an overlapping line at index 7: [[-26.          1.9722221]]
deleting an overlapping line at index 8: [[-5.         1.9373155]]
deleting an overlapping line at index 8: [[-5.         1.9373155]]
Overlapping indexes: [4, 9, 7, 8, 8]


In [12]:
perimiter_lines = []
for i in range(len(lines)):
    if i not in overlapping_indexes:
        perimiter_lines.append(lines[i])
        
print(len(perimiter_lines))

6


In [19]:
for line in perimiter_lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(og_img, (x1, y1), (x2, y2), (0, 0, 255), 2)

In [20]:
showImage(og_img)